In [1]:
import numpy as np
import open3d as o3d
import copy
import math

from collections import defaultdict
from scipy.spatial import Delaunay
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
class HalfEdge:
    def __init__(self):
        self.origin = None  # Начальная вершина
        self.twin = None    # Парное полурёбро
        self.face = None    # Принадлежащая грань
        self.next = None    # Следующее полурёбро в грани

    def __repr__(self):
        return f"HalfEdge({self.origin}, {self.twin.origin})"
        

class Face:
    def __init__(self):
        self.edge = None  # Любое граничное полурёбро

    def vertex_indices(self):
        if self.edge is None:
            return
        
        current = self.edge
        while True:
            yield current.origin
            current = current.next
            
            if current is self.edge:
                break

class DCEL:
    def __init__(self, vertices, edges, faces):
        self.vertices = vertices
        self.edges = edges
        self.faces = faces

In [3]:
# Хеш-функция для вершин (учет возможных погрешностей float)
def vertex_hash(v):
    return tuple(np.round(v, 6))

def segment_hash(seg):
    return tuple(sorted(seg))


def repair_mesh(mesh):
    # Проверяем, есть ли проблема с вершинами
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Создаем словарь для устранения дубликатов вершин
    vertex_map = {}
    new_vertices = []
    new_triangles = []

    # Строим отображение старых вершин в новые
    for idx, v in enumerate(vertices):
        v_hash = vertex_hash(v)
        if v_hash not in vertex_map:
            vertex_map[v_hash] = len(new_vertices)
            new_vertices.append(v)

    # Перестраиваем треугольники с новыми индексами
    for tri in triangles:
        new_tri = [
            vertex_map[vertex_hash(vertices[tri[0]])],
            vertex_map[vertex_hash(vertices[tri[1]])],
            vertex_map[vertex_hash(vertices[tri[2]])]
        ]
        new_triangles.append(new_tri)

    # Создаем новую сетку
    repaired_mesh = o3d.geometry.TriangleMesh()
    repaired_mesh.vertices = o3d.utility.Vector3dVector(np.array(new_vertices))
    repaired_mesh.triangles = o3d.utility.Vector3iVector(
        np.array(new_triangles)
    )

    # Вычисляем нормали для корректного отображения
    repaired_mesh.compute_vertex_normals()

    return repaired_mesh


def load_and_repair_mesh(filepath):
    mesh = o3d.io.read_triangle_mesh(filepath)
    repaired_mesh = repair_mesh(mesh)

    return repaired_mesh

In [4]:
mesh_a = load_and_repair_mesh("model.stl")
mesh_b = load_and_repair_mesh("sphere.stl")

if not mesh_a.has_vertex_normals():
    mesh_a.compute_vertex_normals()
if not mesh_b.has_vertex_normals():
    mesh_b.compute_vertex_normals()

In [5]:
def triangle_area(v1, v2, v3):
    """Вычисление площади треугольника"""
    return 0.5 * np.linalg.norm(np.cross(v2 - v1, v3 - v1))

def center_of_mass(mesh):
    """Вычисление центра масс меша"""
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    
    # Получаем вершины каждого треугольника
    tri_vertices = vertices[triangles]
    
    # Центры треугольников
    centers = np.mean(tri_vertices, axis=1)
    
    # Площади треугольников
    areas = np.array([triangle_area(t[0], t[1], t[2]) for t in tri_vertices])
    
    # Центр масс
    return np.sum(centers * areas.reshape(-1, 1), axis=0) / np.sum(areas)

In [6]:
ITERS = 10

def normalize(vector):
    if not np.any(vector):
        raise ZeroDivisionError

    return vector / np.linalg.norm(vector)


def get_adjacent_vertices(mesh):
    """Построение словаря смежных вершин"""
    triangles = np.asarray(mesh.triangles)
    adjacency_dict = defaultdict(set)

    for tri in triangles:
        v0, v1, v2 = tri

        # Добавляем связи между вершинами треугольника
        adjacency_dict[v0].update([v1, v2])
        adjacency_dict[v1].update([v0, v2])
        adjacency_dict[v2].update([v0, v1])

    return adjacency_dict

def check_orientation(tri_vertices):
    return np.dot(np.cross(tri_vertices[1] - tri_vertices[0], tri_vertices[2] - tri_vertices[0]), tri_vertices[0]) >= 0

def relax_mesh(mesh, eps=1e-2):
    """Релаксация меша с сохранением топологии"""
    # Получаем копию исходного меша
    relaxed_mesh = copy.deepcopy(mesh)
    
    # Получаем вершины и треугольники
    vertices = np.asarray(relaxed_mesh.vertices)
    triangles = np.asarray(relaxed_mesh.triangles)
    adj = get_adjacent_vertices(mesh)
    
    # Релаксация
    while True:
        prev_vertices = copy.deepcopy(vertices)
        
        for i in range(len(vertices)):
            neighbors = list(adj[i])
            vertices[i] = normalize(np.mean(prev_vertices[neighbors], axis=0))
        
        if not np.all(map(lambda tri: check_orientation(vertices[tri]), triangles)):
            continue
        
        if np.allclose(prev_vertices, vertices, rtol=eps):
            break

    # Обновляем нормали
    relaxed_mesh.compute_vertex_normals()
    return relaxed_mesh


def parametrize_mesh(mesh):
    """Параметризация меша на сфере"""
    # Создаем копию меша
    mesh_copy = copy.deepcopy(mesh)

    # Вычисляем центр масс
    center = center_of_mass(mesh_copy)

    # Переносим вершины в начало координат и нормализуем
    vertices = np.asarray(mesh_copy.vertices)
    vertices -= center
    norms = np.linalg.norm(vertices, axis=1)
    norms[norms == 0] = 1  # Избегаем деления на ноль #TODO: troubleshoot
    vertices = vertices / norms.reshape(-1, 1)

    # Обновляем вершины и нормали
    mesh_copy.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_copy.compute_vertex_normals()

    # Применяем релаксацию
    return relax_mesh(mesh_copy)

In [7]:
def overlap_meshes(source_mesh, target_mesh):
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = source_mesh.vertices
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = target_mesh.vertices
    
    source_pcd.estimate_normals()
    target_pcd.estimate_normals()

    trans_init = np.identity(4)
    threshold = 0.02 * 2
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    source_pcd.transform(reg_p2l.transformation)
    source_mesh.vertices = source_pcd.points

In [8]:
def add_wireframe_to_geo(mesh, geo, color = [0, 0, 0]):
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices

    pcd.paint_uniform_color(color)
    wireframe.paint_uniform_color(color)

    geo.append(wireframe)
    geo.append(pcd)


def add_dcell_to_geo(dcel, geo, color = [0, 0, 0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(dcel.vertices)
    pcd.paint_uniform_color(color)

    lineset = o3d.geometry.LineSet()
    lineset.points = pcd.points
    lineset.lines = o3d.utility.Vector2iVector(np.array([[e.origin, e.twin.origin] for e in dcel.edges]))
    lineset.paint_uniform_color(color)

    geo.append(lineset)
    geo.append(pcd)

In [ ]:
def segments(mesh):
    seen = set()

    for tri in np.asarray(mesh.triangles):
        segments = (
            tuple(sorted([tri[0], tri[1]])),
            tuple(sorted([tri[1], tri[2]])),
            tuple(sorted([tri[2], tri[0]]))
        )
        for seg in segments:
            if seg not in seen:
                seen.add(seg)
                yield seg


def arcs(mesh):
    vertices = np.asarray(mesh.vertices)


    for seg in segments(mesh):
        yield (vertices[seg[0]], vertices[seg[1]])


def find_arcs_intersection(arc_a, arc_b):
    normal_a = np.cross(arc_a[0], arc_a[1])
    if np.dot(arc_a[0], arc_b[0]) < 0:
        return None

    normal_b = np.cross(arc_b[0], arc_b[1])
    d = np.cross(normal_a, normal_b)

    if np.isclose(np.linalg.norm(d), 0):
        return None

    if np.dot(d, arc_a[0]) < 0:
        d = -d

    if np.dot(np.cross(arc_a[0], d), np.cross(arc_a[1], d)) >= 0:
        return None

    if np.dot(np.cross(arc_b[0], d), np.cross(arc_b[1], d)) >= 0:
        return None

    return normalize(d)

In [68]:
def mesh_from_dcel(dcel: DCEL):
    triangles = []
    vertices = dcel.vertices_coords()
    
    for face in dcel.faces: 
        triangles.append(list(face.vertex_indices()))

    pprint(triangles)

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(np.array(triangles))
    mesh.compute_vertex_normals()

    return mesh

In [ ]:
def tangent_angle(normal, direction):
    """Вычисляет угол направления в касательной плоскости вершины."""
    # Шаг 1: Выбираем опорный вектор, не коллинеарный нормали
    if abs(normal[0]) < 0.1 and abs(normal[1]) < 0.1:
        ref = (1, 0, 0)  # Если нормаль близка к оси Z
    else:
        ref = (0, 0, 1)
    
    # Шаг 2: Строим ортонормированный базис (u, v)
    u = np.cross(normal, ref)
    u = normalize(u)
    
    # Если u нулевой, пробуем другой ref
    if math.sqrt(u[0]**2 + u[1]**2 + u[2]**2) < 1e-6:
        ref = (1, 0, 0) if abs(normal[0]) < 0.5 else (0, 1, 0)
        u = np.cross(normal, ref)
        u = normalize(u)
    
    v = np.cross(normal, u)  # Уже ортонормирован
    
    # Шаг 3: Проекция вектора направления
    w_proj = direction  # Мы не вычитаем нормальную компоненту, т.к. dot(w_proj, v) даст то же
    x = np.dot(w_proj, u)
    y = np.dot(w_proj, v)
    
    # Шаг 4: Вычисляем угол [0, 2π]
    angle = math.atan2(y, x)
    return angle + 2*math.pi if angle < 0 else angle


def build_dcel(vertices, segments):
    vertex_objs = np.array(vertices)
    edges = np.ndarray((0, ), dtype=HalfEdge)
    faces = np.ndarray((0, ), dtype=Face)
    edge_map = defaultdict(list)

    for start, end in segments:
        he1 = HalfEdge()
        he2 = HalfEdge()

        he1.twin = he2
        he2.twin = he1

        he1.origin = start
        he2.origin = end

        edges = np.append(edges, [he1, he2])

        edge_map[start].append(he1)
        edge_map[end].append(he2)

    for key in edge_map:
        edge_map[key].sort(key=lambda he: tangent_angle(vertices[key], vertices[he.twin.origin] - vertices[key]))

    for outgoing_edges in edge_map.values():
        n = len(outgoing_edges)
        if n < 2:
            raise ValueError("Invalid DCEL. There should be at least two half edges for a vertex")
        
        for j in range(n):
            outgoing_edges[j].twin.next = outgoing_edges[(j + 1) % n]

    for he in tqdm(edges):
        if he.face is not None:
            continue
        
        face = Face()
        face.edge = he
        faces = np.append(faces, [face])
        
        current = he
        while True:
            current.face = face
            current = current.next
            if current is he:
                break

    return DCEL(vertex_objs, edges, faces)

In [81]:
class Vertex:
    def __init__(self, index, coords):
        self.index = index # Original index from input list
        self.coords = np.array(coords, dtype=float) # 3D coordinates as numpy array
        self.leaving_edge = None # One outgoing half-edge originating from this vertex

    def __repr__(self):
        return f"Vertex(idx={self.index}, coords={self.coords.tolist()})"

class HalfEdge:
    def __init__(self):
        self.origin = None      # Vertex object: starting vertex of this half-edge
        self.twin = None        # HalfEdge object: oppositely oriented twin
        self.face = None        # Face object: incident face (to the left of the edge, following its direction)
        self.next = None        # HalfEdge object: next half-edge in the CCW cycle of self.face
        self.prev = None        # HalfEdge object: previous half-edge in the CCW cycle of self.face
        # self._id = id(self) # Optional: for easier debugging if needed

    def __repr__(self):
        orig_idx = self.origin.index if self.origin else "None"
        # The destination of this half-edge is the origin of its twin
        dest_idx = self.twin.origin.index if self.twin and self.twin.origin else "None"
        face_id_str = f"F_id:{id(self.face)}" if self.face else "NoFace"
        return f"HalfEdge(V{orig_idx} -> V{dest_idx}, {face_id_str})"

class Face:
    def __init__(self):
        self.edge = None # One of the half-edges bounding this face
        # self._id = id(self) # Optional: for easier debugging

    def __repr__(self):
        if not self.edge or not self.edge.origin:
            return f"Face(empty_or_invalid, id={id(self)})"
        # Gather vertex indices for a more informative representation
        try:
            indices = self.vertex_indices()
            return f"Face(V_indices={indices}, id={id(self)})"
        except Exception: # Catch potential errors during traversal if structure is malformed
            return f"Face(edge_from_V{self.edge.origin.index if self.edge and self.edge.origin else 'N/A'}, id={id(self)}, error_in_repr)"


    def vertex_indices(self): 
        """Returns a list of original indices of vertices forming this face, in CCW order."""
        if self.edge is None:
            return []
        
        path_indices = []
        current = self.edge
        max_verts_in_face = 1000 # Safety break for malformed structures or very large faces
        
        for _ in range(max_verts_in_face):
            if current is None or current.origin is None:
                # print(f"Warning: Malformed face structure encountered in vertex_indices() for face {id(self)}.")
                break 
            path_indices.append(current.origin.index)
            current = current.next
            if current is self.edge: # Back to start
                break
        else: # Loop finished without break (max_verts_in_face reached)
            # print(f"Warning: Exceeded max_verts_in_face ({max_verts_in_face}) in Face.vertex_indices() for face {id(self)}.")
            pass # Return whatever was collected
        return path_indices

    def vertices(self): 
        """Returns a list of Vertex objects forming this face, in CCW order."""
        if self.edge is None:
            return []
        
        path_vertices = []
        current = self.edge
        max_verts_in_face = 1000 # Safety break

        for _ in range(max_verts_in_face):
            if current is None or current.origin is None:
                # print(f"Warning: Malformed face structure encountered in vertices() for face {id(self)}.")
                break
            path_vertices.append(current.origin)
            current = current.next
            if current is self.edge:
                break
        else: # Loop finished without break
            # print(f"Warning: Exceeded max_verts_in_face ({max_verts_in_face}) in Face.vertices() for face {id(self)}.")
            pass
        return path_vertices

class DCEL:
    def __init__(self, vertices, half_edges, faces):
        self.vertices = vertices     # List of Vertex objects
        self.half_edges = half_edges # List of HalfEdge objects
        self.faces = faces           # List of Face objects

    def __repr__(self):
        return f"DCEL(V:{len(self.vertices)}, HE:{len(self.half_edges)}, F:{len(self.faces)})"

    def vertices_coords(self):
        return np.fromiter(map(lambda v: v.coords, self.vertices), dtype=np.ndarray)

def tangent_angle_around_vertex_normal(vertex_pos_vec_raw, edge_direction_vec_raw):
    """
    Calculates the angle of an edge_direction_vec in the plane perpendicular to vertex_pos_vec_raw.
    Assumes vertex_pos_vec_raw (from origin to vertex) acts as the outward normal.
    This defines a CCW ordering when looking *against* vertex_pos_vec_raw.
    Args:
        vertex_pos_vec_raw (np.array): The 3D coordinate of the vertex (serves as normal).
        edge_direction_vec_raw (np.array): The direction vector of the outgoing edge.
    Returns:
        float: Angle in radians [0, 2*pi). Returns 0 if normal is zero.
    """
    normal = normalize(np.array(vertex_pos_vec_raw))
    direction = normalize(np.array(edge_direction_vec_raw))

    if np.linalg.norm(normal) < 1e-9: # Vertex is at origin, normal is undefined here
        # print("Warning: Vertex at origin, tangent_angle cannot be reliably computed based on position vector.")
        return 0.0 # Arbitrary angle, sorting will be arbitrary for this vertex's edges

    # Choose a reference vector for 0 angle, not collinear with normal
    # Try X-axis first
    ref_axis_candidate = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(normal, ref_axis_candidate)) > 0.99: # If normal is too close to X-axis
        ref_axis_candidate = np.array([0.0, 1.0, 0.0]) # Try Y-axis
        if abs(np.dot(normal, ref_axis_candidate)) > 0.99: # If normal is also too close to Y-axis
             ref_axis_candidate = np.array([0.0, 0.0, 1.0]) # Try Z-axis

    # Project ref_axis_candidate onto the plane perpendicular to normal to get u_axis (local x-axis)
    u_axis = ref_axis_candidate - np.dot(ref_axis_candidate, normal) * normal
    u_axis = normalize(u_axis)
    
    # If u_axis is zero (e.g. ref_axis_candidate was collinear with normal despite checks, or normal was zero)
    if np.linalg.norm(u_axis) < 1e-9:
        # This case should be rare if normal is not zero and ref_axis_candidate selection is robust.
        # Fallback: try to create a perpendicular vector directly if normal is not zero.
        if abs(normal[0]) > 0.1 or abs(normal[1]) > 0.1 : # if normal is not (0,0,Z)
            u_axis = normalize(np.array([-normal[1], normal[0], 0]))
        else: # if normal is (0,0,Z) like
            u_axis = normalize(np.array([1,0,0]))

        if np.linalg.norm(u_axis) < 1e-9: # Still zero, something is very wrong or normal was zero
             # print(f"Warning: u_axis is zero vector. Normal: {normal.tolist()}")
             return 0.0 # Arbitrary angle

    # v_axis (local y-axis) is normal x u_axis (forms a right-handed system: u, v, normal if normal is Z)
    # We want CCW angle when looking AGAINST normal. So (normal, u_axis, v_axis) should be RH.
    # Or, (u_axis, v_axis) define the plane, and atan2(dot(dir,v), dot(dir,u)) gives angle.
    v_axis = np.cross(normal, u_axis) 
    # v_axis should already be normalized if normal and u_axis are ortho-normalized.

    x_comp = np.dot(direction, u_axis)
    y_comp = np.dot(direction, v_axis)
    
    angle = math.atan2(y_comp, x_comp) # Angle in [-pi, pi]
    return angle if angle >= 0 else angle + 2 * math.pi # Map to [0, 2pi]

def build_dcel(vertex_coords_list, segments_vertex_indices):
    """
    Builds a DCEL from a list of 3D vertex coordinates and a list of segments (edges).
    ASSUMPTIONS:
    1. The segments form a 2-manifold mesh.
    2. The mesh is star-shaped with respect to the origin (0,0,0).
    3. The origin (0,0,0) is INSIDE the mesh.
    4. No vertex is located exactly at the origin (0,0,0) for reliable orientation.
    These assumptions allow using vertex position vectors as outward normals for sorting edges
    cyclically, aiming for CCW face windings when viewed from outside.

    Args:
        vertex_coords_list (list): List of 3D coordinates (e.g., [[x1,y1,z1], [x2,y2,z2], ...]).
        segments_vertex_indices (list): List of pairs of vertex indices (e.g., [[0,1], [1,2], ...]).
    """
    
    # 1. Create Vertex objects
    vertex_objs = [Vertex(i, coords) for i, coords in enumerate(vertex_coords_list)]

    all_half_edges = []
    # edge_map: maps vertex_index to a list of outgoing HalfEdges originating from it
    edge_map = defaultdict(list) 

    # 2. Create HalfEdges for each segment
    for start_idx, end_idx in segments_vertex_indices:
        if start_idx == end_idx:
            # print(f"Warning: Segment from vertex {start_idx} to itself found. Skipping.")
            continue
        if not (0 <= start_idx < len(vertex_objs) and 0 <= end_idx < len(vertex_objs)):
            # print(f"Warning: Segment indices ({start_idx}, {end_idx}) out of bounds. Skipping.")
            continue

        he1 = HalfEdge() # From start_idx to end_idx
        he2 = HalfEdge() # From end_idx to start_idx (twin)

        he1.twin = he2
        he2.twin = he1

        he1.origin = vertex_objs[start_idx]
        he2.origin = vertex_objs[end_idx]
        
        if vertex_objs[start_idx].leaving_edge is None:
            vertex_objs[start_idx].leaving_edge = he1
        if vertex_objs[end_idx].leaving_edge is None:
            vertex_objs[end_idx].leaving_edge = he2

        all_half_edges.extend([he1, he2])

        edge_map[start_idx].append(he1) # he1 is outgoing from start_idx
        edge_map[end_idx].append(he2)   # he2 is outgoing from end_idx

    # 3. Sort outgoing HalfEdges around each vertex
    for vertex_idx, outgoing_hes_at_vertex in edge_map.items():
        current_vertex_obj = vertex_objs[vertex_idx]
        
        # The "normal" for sorting is the vector from origin to the current vertex.
        # This relies on the star-shaped assumption for CCW-from-outside orientation.
        vertex_position_as_normal = current_vertex_obj.coords 
        
        if np.linalg.norm(vertex_position_as_normal) < 1e-9:
             # print(f"Warning: Vertex {vertex_idx} is at the origin. Edge sorting around it may be arbitrary.")
             pass # Sorting will use angle 0.0 from tangent_angle function

        def sort_key_func(he_to_sort):
            # he_to_sort.origin is current_vertex_obj
            # he_to_sort.twin.origin is the destination vertex of he_to_sort
            if he_to_sort.twin is None or he_to_sort.twin.origin is None:
                # print(f"Warning: Malformed half-edge or twin for sorting at vertex {vertex_idx}. Assigning max angle.")
                return 2 * math.pi # Put problematic edges at the end
            
            edge_direction_vector = he_to_sort.twin.origin.coords - current_vertex_obj.coords
            return tangent_angle_around_vertex_normal(vertex_position_as_normal, edge_direction_vector)

        outgoing_hes_at_vertex.sort(key=sort_key_func, reverse=1)

    # 4. Set `next` and `prev` pointers for HalfEdges based on sorted order
    for vertex_idx, sorted_outgoing_hes in edge_map.items():
        num_outgoing = len(sorted_outgoing_hes)
        if num_outgoing == 0: 
            continue # No edges outgoing from this vertex (isolated vertex or error)
        
        for i in range(num_outgoing):
            # he_current_sorted is an edge *leaving* vertex_idx
            he_current_sorted = sorted_outgoing_hes[i]
            # he_next_sorted_around_vertex is the *next* edge leaving vertex_idx in CCW order
            he_next_sorted_around_vertex = sorted_outgoing_hes[(i + 1) % num_outgoing]

            # he_current_sorted.twin is an edge *entering* vertex_idx.
            # Its `next` pointer in its face cycle should be he_next_sorted_around_vertex.
            if he_current_sorted.twin is None:
                # print(f"Warning: Missing twin for edge originating at {he_current_sorted.origin.index if he_current_sorted.origin else 'N/A'} at vertex {vertex_idx}. Cannot set next/prev.")
                continue

            he_current_sorted.twin.next = he_next_sorted_around_vertex
            he_next_sorted_around_vertex.prev = he_current_sorted.twin
            
    # 5. Discover faces and assign `face` pointers to HalfEdges
    discovered_faces = []
    # Can use tqdm(all_half_edges) if progress bar is desired
    for he_candidate_start_face in all_half_edges:
        if he_candidate_start_face.face is None: # If this half-edge hasn't been assigned to a face
            new_face = Face()
            new_face.edge = he_candidate_start_face 
            
            current_he_in_face = he_candidate_start_face
            face_edge_count = 0 
            max_edges_per_face = len(all_half_edges) + 1 # Safety break

            valid_face = True
            temp_face_edges = [] # Store edges for this potential face

            while True:
                if current_he_in_face is None: # Should not happen if next/prev are set correctly
                    # print(f"Error: Encountered None HalfEdge during face traversal for potential face starting near V{he_candidate_start_face.origin.index if he_candidate_start_face.origin else 'N/A'}.")
                    valid_face = False
                    break
                if current_he_in_face.face is not None and current_he_in_face.face != new_face : # Edge already belongs to another face
                    # This indicates a non-manifold structure or an error in linking
                    # print(f"Error: Edge {current_he_in_face} already belongs to another face {id(current_he_in_face.face)} while trying to form new face {id(new_face)}.")
                    valid_face = False
                    break
                
                current_he_in_face.face = new_face
                temp_face_edges.append(current_he_in_face)
                
                current_he_in_face = current_he_in_face.next
                face_edge_count += 1

                if current_he_in_face is he_candidate_start_face: # Cycled back to the start
                    break 
                if face_edge_count > max_edges_per_face:
                    # print(f"Error: Face traversal exceeded max_edges_per_face for potential face starting near V{he_candidate_start_face.origin.index if he_candidate_start_face.origin else 'N/A'}. Face may be malformed.")
                    valid_face = False
                    break
            
            if valid_face and face_edge_count > 0 : # Minimum 1 edge (though faces usually have >=3)
                discovered_faces.append(new_face)
            else:
                # Rollback face assignment for edges in the failed attempt
                for edge_in_failed_face in temp_face_edges:
                    if edge_in_failed_face.face == new_face:
                        edge_in_failed_face.face = None 
                # print(f"Info: Discarded invalid or empty face starting with edge {he_candidate_start_face}")


    return DCEL(vertex_objs, all_half_edges, discovered_faces)

In [82]:
def create_dcel_map(mesh_a, mesh_b):
    segment_map = defaultdict(set)

    vertices = np.concat((np.asarray(mesh_a.vertices), np.asarray(mesh_b.vertices)))
    segments_ = []

    for seg_a in tqdm(segments(mesh_a), desc="Intersecting meshes"):
        for seg_b in segments(mesh_b):
            seg_b = (seg_b[0] + len(mesh_a.vertices),
                     seg_b[1] + len(mesh_a.vertices))
            arc_inter = find_arcs_intersection(
                (vertices[seg_a[0]], vertices[seg_a[1]]),
                (vertices[seg_b[0]], vertices[seg_b[1]]),
            )

            segment_map[segment_hash(seg_a)].update((seg_a[0], seg_a[1]))
            segment_map[segment_hash(seg_b)].update((seg_b[0], seg_b[1]))

            if arc_inter is not None:
                vertices = np.append(vertices, [arc_inter], axis=0)
                segment_map[segment_hash(seg_a)].add(len(vertices) - 1)
                segment_map[segment_hash(seg_b)].add(len(vertices) - 1)

    for (start_idx, _), points in segment_map.items():
        points = sorted(points, key=lambda i: np.linalg.norm(np.cross(vertices[start_idx], vertices[i])))
        for i in range(len(points) - 1):
            segments_.append([points[i], points[i + 1]])

    # remove duplicate vertices
    vertices_map = {}
    new_vertices = []
    new_segments = set()

    for v in vertices:
        v_hash = vertex_hash(v)
        if v_hash not in vertices_map:
            vertices_map[v_hash] = len(new_vertices)
            new_vertices.append(v)

    for start, end in segments_:
        seg = (
            vertices_map[vertex_hash(vertices[start])],
            vertices_map[vertex_hash(vertices[end])]
        )

        if seg[0] == seg[1]:
            continue

        new_segments.add(segment_hash(seg))

    return build_dcel(new_vertices, new_segments)

In [74]:
def triangulate_face(face_vertices):
    """
    Триангулирует плоскую грань многогранника с использованием триангуляции Делоне.

    Аргументы:
        face_vertices (np.array): Массив вершин грани в 3D-пространстве.
                                  Формат: [[x1, y1, z1], [x2, y2, z2], ...]

    Возвращает:
        np.array: Массив индексов треугольников. Каждый треугольник представлен
                  тремя индексами вершин из исходного массива face_vertices.
                  Формат: [[idx1, idx2, idx3], ...]
    """
    if len(face_vertices) < 3:
        print("Ошибка: Для триангуляции требуется как минимум 3 вершины.")
        return np.array([])

    # 1. Находим центр масс грани
    center = np.mean(face_vertices, axis=0)

    # 2. Вычисляем нормальный вектор грани
    # Для плоской грани, нормаль можно найти, используя первые три вершины (если их достаточно)
    # или более надежным способом, усредняя нормали от соседних треугольников.
    # Здесь мы используем первые три вершины для простоты, предполагая, что они не коллинеарны.
    v0 = face_vertices[0]
    v1 = face_vertices[1]
    v2 = face_vertices[2]

    # Два вектора в плоскости грани
    vec1 = v1 - v0
    vec2 = v2 - v0

    # Нормальный вектор (векторное произведение)
    normal = np.cross(vec1, vec2)
    # Нормализуем нормальный вектор
    norm_length = np.linalg.norm(normal)
    if norm_length == 0:
        print("Ошибка: Вершины грани коллинеарны или совпадают, невозможно определить нормаль.")
        return np.array([])
    normal = normal / norm_length

    # 3. Создаем ортонормированный базис для плоскости грани
    # Выбираем первый вектор базиса (u_vec) как любой вектор в плоскости,
    # ортогональный нормали. Например, проецируем произвольный вектор на плоскость.
    # Произвольный вектор, не параллельный нормали
    arbitrary_vec = np.array([1.0, 0.0, 0.0])
    if np.dot(arbitrary_vec, normal) > 0.99: # Если почти параллелен нормали, выбираем другой
        arbitrary_vec = np.array([0.0, 1.0, 0.0])

    # Проецируем произвольный вектор на плоскость (т.е. ортогонально нормали)
    u_vec = arbitrary_vec - np.dot(arbitrary_vec, normal) * normal
    u_length = np.linalg.norm(u_vec)
    if u_length == 0:
        print("Ошибка: Не удалось создать первый базисный вектор.")
        return np.array([])
    u_vec = u_vec / u_length

    # Второй вектор базиса (v_vec) ортогонален u_vec и normal
    v_vec = np.cross(normal, u_vec)
    # v_vec уже должен быть нормализован, но на всякий случай
    v_vec = v_vec / np.linalg.norm(v_vec)

    # 4. Проецируем 3D-вершины на 2D-плоскость, используя новый базис
    projected_points_2d = []
    for v in face_vertices:
        # Вектор от центра грани до вершины
        vec_from_center = v - center
        # Проекция на u_vec и v_vec дает 2D-координаты
        x_2d = np.dot(vec_from_center, u_vec)
        y_2d = np.dot(vec_from_center, v_vec)
        projected_points_2d.append([x_2d, y_2d])

    projected_points_2d = np.array(projected_points_2d)

    # 5. Выполняем триангуляцию Делоне на 2D-точках
    tri = Delaunay(projected_points_2d)
    return tri.simplices

In [ ]:
def create_super_mesh(mesh_a, mesh_b):
    dcel_map = create_dcel_map(mesh_a, mesh_b)
    dcel_vertices = dcel_map.vertices_coords()

    triangles = np.ndarray((0, 3), dtype=np.int32)
    for face in dcel_map.faces:
        vertex_indices = np.fromiter(face.vertex_indices(), dtype=int)
        face_vertices = dcel_vertices[vertex_indices]
        face_triangles = vertex_indices[triangulate_face(face_vertices)]
        triangles = np.append(triangles, face_triangles, axis=0)

    super_mesh = o3d.geometry.TriangleMesh()
    super_mesh.vertices = o3d.utility.Vector3dVector(dcel_vertices)
    super_mesh.triangles = o3d.utility.Vector3iVector(triangles)
    super_mesh.compute_vertex_normals()
    
    return super_mesh

In [15]:
def compute_barycentric(P, A, B, C):
    v0 = B - A
    v1 = C - A
    v2 = P - A
    
    cross_ABC = np.cross(v0, v1)
    denom = np.dot(cross_ABC, cross_ABC)
    
    # Относительные площади с учётом знака
    v = np.dot(np.cross(v2, v1), cross_ABC) / denom
    w = np.dot(np.cross(v0, v2), cross_ABC) / denom
    u = 1 - v - w
    
    return np.array([u, v, w])

def find_enclosing_triangle(point, mesh):
    vertices = np.asarray(mesh.vertices)

    for tri in np.asarray(mesh.triangles):
        tri_vertices = vertices[tri]

        # Project point on triangle's tangent plane
        normal = np.cross(tri_vertices[1] - tri_vertices[0], tri_vertices[2] - tri_vertices[0])
        t = np.dot(normal, tri_vertices[0]) / np.dot(normal, point)
        
        if not 0 < t < 1.01:
            continue
        
        projected_point = point * t
        bary = compute_barycentric(projected_point, *tri_vertices)

        if np.all(bary >= -1e-6):
            return tri, bary

    raise RuntimeError("Could not find any correspoinding triangle which is impossible")


In [16]:
parametrized_mesh_a = parametrize_mesh(mesh_a)
parametrized_mesh_b = parametrize_mesh(mesh_b)

In [83]:
overlap_meshes(parametrized_mesh_b, parametrized_mesh_a) #TODO: don't change mesh_b
super_mesh = create_super_mesh(parametrized_mesh_a, parametrized_mesh_b)

Intersecting meshes: 384it [00:41,  9.34it/s]


In [85]:
source_vertices = np.asarray(mesh_a.vertices)
target_vertices = np.asarray(mesh_b.vertices)

v_source = np.zeros((len(super_mesh.vertices), 3))
v_target = np.zeros((len(super_mesh.vertices), 3))

for i, v in tqdm(enumerate(np.asarray(super_mesh.vertices)), total=len(super_mesh.vertices), desc="Interpolating super-mesh to original and target meshes"):
    src_tri, src_bary = find_enclosing_triangle(v, parametrized_mesh_a)
    v_source[i] = np.dot(src_bary, source_vertices[src_tri])

    tgt_tri, tgt_bary = find_enclosing_triangle(v, parametrized_mesh_b)
    v_target[i] = np.dot(tgt_bary, target_vertices[tgt_tri])


Interpolating super-mesh to original and target meshes: 100%|██████████| 2103/2103 [00:29<00:00, 71.95it/s] 


In [86]:
def create_interpolation(source_vertices, taget_vertices):
    
    def interp(t):
        return (1 - t) * source_vertices + t * taget_vertices
    
    return interp

In [87]:
interp = create_interpolation(v_source, v_target)
result = copy.deepcopy(super_mesh)

def update_mesh(t):
    global result
    result.vertices = o3d.utility.Vector3dVector(interp(t))
    result.compute_vertex_normals()

update_mesh(0)

In [92]:
t = 0
t_step = 0.02

vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()
vis.add_geometry(result)


def increase_t(vis):
    global t, t_step, result
    t = min(t + t_step, 1.0)
    current_transform = vis.get_view_control().convert_to_pinhole_camera_parameters().extrinsic
    update_mesh(t)
    vis.clear_geometries()
    vis.add_geometry(result)
    camera_params = vis.get_view_control().convert_to_pinhole_camera_parameters()
    camera_params.extrinsic = current_transform
    vis.get_view_control().convert_from_pinhole_camera_parameters(camera_params)

def decrease_t(vis):
    global t, t_step, result
    t = max(t - t_step, 0.0)
    current_transform = vis.get_view_control().convert_to_pinhole_camera_parameters().extrinsic
    update_mesh(t)
    vis.clear_geometries()
    vis.add_geometry(result)
    camera_params = vis.get_view_control().convert_to_pinhole_camera_parameters()
    camera_params.extrinsic = current_transform
    vis.get_view_control().convert_from_pinhole_camera_parameters(camera_params)

vis.register_key_callback(ord('A'), decrease_t)
vis.register_key_callback(ord('D'), increase_t)

vis.run()

In [22]:
geo = []

update_mesh(1)
# add_wireframe_to_geo(super_mesh, geo, [1, 0, 0])
add_wireframe_to_geo(result, geo)

o3d.visualization.draw_geometries(geo + [result])

In [23]:
geo = []

add_wireframe_to_geo(parametrized_mesh_a, geo, [0, 0, 1])
add_wireframe_to_geo(parametrized_mesh_b, geo, [1, 0, 0])
# add_wireframe_to_geo(super_mesh, geo)

o3d.visualization.draw_geometries(geo, mesh_show_wireframe=True)